In [1]:
import os
import glob
import pandas as pd
import numpy as np
from collections import Counter,defaultdict
import re
import pickle
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_pickle('../data_scraping/dedup_combined_df.pkl')
df.head(10)

,url,title,date,domain,stance,topic,is_AP,year,pretty_domain,month
0,www.buzzfeednews.com/article/tasneemnashrulla/...,"""eat the babies"" viral video at aoc town hall ...",2019-10-04 00:00:00,buzzfeed,pro,cc,None,2019.0,Buzzfeed,10.0
1,www.buzzfeednews.com/article/passantino/extrem...,"""extremely likely"" global warming is man-made,...",2013-09-27 00:00:00,buzzfeed,pro,cc,None,2013.0,Buzzfeed,9.0
2,shotofprevention.com/2010/11/03/history-makes-...,"""history"" makes headlines with launch of new w...",2020-03-13 14:32:02,https://shotofprevention/,pro,vax,False,2020.0,Shot of Prevention,3.0
3,www.buzzfeednews.com/article/andrewkaczynski/i...,"""it's global warming, stupid"" - buzzfeed news",2012-11-01 00:00:00,buzzfeed,pro,cc,None,2012.0,Buzzfeed,11.0
4,www.buzzfeednews.com/article/tasneemnashrulla/...,"""japan dropped an atomic bomb on america durin...",2014-02-24 00:00:00,buzzfeed,pro,cc,None,2014.0,Buzzfeed,2.0
5,www.buzzfeednews.com/article/llevin/opinion-im...,"""look at my record, child"": joe biden showed m...",2019-10-31 00:00:00,buzzfeed,pro,cc,None,2019.0,Buzzfeed,10.0
6,www.breitbart.com/politics/2019/06/14/orourke-...,"""president o'rourke will end oil and gas lease...",2019-06-14 00:00:00,breitbart,anti,cc,None,2019.0,Breitbart,6.0
7,www.buzzfeednews.com/article/andrewkaczynski/s...,smoking doesnt kill and other great old opeds ...,2015-03-31 00:00:00,buzzfeed,pro,cc,None,2015.0,Buzzfeed,3.0
8,www.foxnews.com/world/100-carbon-tax-by-2030-c...,"$100 carbon tax by 2030 could save climate, sa...",2017-05-29 00:00:00,fox,anti,cc,None,2017.0,Fox,5.0
9,childrenshealthdefense.org/news/4-billion-and-...,$4 billion and growing: u.s. payouts for vacc...,2018-11-19 00:00:00,chd,anti,vax,None,2018.0,Children's Health Defense,11.0


In [3]:
TOPICS = ['cc','vax','covid']
SIDES = ['pro','anti']

In [4]:
good_quotes = pickle.load(open('../data_processing/good_coref_quotes.pkl','rb'))
covid_quotes = pickle.load(open('../data_processing/filtered_covid_quotes.pkl','rb'))
len(good_quotes),len(covid_quotes)

(434634, 223227)

In [5]:
cc_quotes = [q for q in good_quotes if q['topic'] == 'cc']
vax_quotes = [q for q in good_quotes if q['topic'] == 'vax']
quotes_by_topic_side = {'cc':{'pro':[q for q in cc_quotes if q['stance']=='pro'],
                         'anti':[q for q in cc_quotes if q['stance']=='anti']},
                   'vax':{'pro':[q for q in vax_quotes if q['stance']=='pro'],
                         'anti':[q for q in vax_quotes if q['stance']=='anti']},
                   'covid':{'pro':[q for q in covid_quotes if q['stance']=='pro'],
                         'anti':[q for q in covid_quotes if q['stance']=='anti']}}

In [6]:
len(cc_quotes)

400460

# Prep for running Dora's code

In [7]:
# Write quote strings to file for Topic analysis

In [8]:
cc_quotes[0].keys()

dict_keys(['quote lemmas', 'quote lemmas coref', 'verb lemmas', 'verb lemmas coref', 'main verb lemma', 'main verb lemma coref', 'subject lemmas', 'subject lemmas coref', 'main subject lemma', 'main subject lemma coref', 'neg lemmas', 'main neg lemma', 'quote text', 'verb text', 'main verb text', 'subject text', 'main subject text', 'is neg', 'source', 'stance', 'topic'])

In [10]:
guid_cc_quotes = []
for ix,q in enumerate(cc_quotes):
    q.update({"guid": 'cc_{}'.format(ix)})
    guid_cc_quotes.append(q)
len(cc_quotes),len(guid_cc_quotes)

(400460, 400460)

In [12]:
pickle.dump(guid_cc_quotes,open('guid_cc_quotes.pkl','wb'))

In [23]:
cc_quotes_df = pd.DataFrame({"quote":[' '.join(q['quote text']) for q in guid_cc_quotes],
                            "guid":[q['guid'] for q in guid_cc_quotes],
                            "source":[q['source'] for q in guid_cc_quotes],
                            "stance":[q['stance'] for q in guid_cc_quotes]})

In [24]:
cc_quotes_df.head()

,quote,guid,source,stance
0,climate change is a hoax,cc_0,www.buzzfeednews.com/article/tasneemnashrulla/...,pro
1,it planted the troll,cc_1,www.buzzfeednews.com/article/tasneemnashrulla/...,pro
2,the stunt was staged by what is now a far - ri...,cc_2,www.buzzfeednews.com/article/tasneemnashrulla/...,pro
3,that Ocasio - Cortez ’s Green New Deal was not...,cc_3,www.buzzfeednews.com/article/tasneemnashrulla/...,pro
4,We ’re not going to be here for much long beca...,cc_4,www.buzzfeednews.com/article/tasneemnashrulla/...,pro


In [26]:
cc_quotes_df.to_csv('cc_quotes.tsv',sep='\t',header=True,index=False)

In [27]:
all_cc_quotes = pd.read_csv('cc_quotes.tsv',sep='\t',header=0)

In [29]:
# Create vocab file for cooccurrence matrix

In [31]:
all_quote_tokens = [x.lower().split() for x in all_cc_quotes.quote.values]

In [38]:
all_vocab = [item for sublist in all_quote_tokens for item in sublist]
len(all_vocab)

6340399

In [33]:
all_vocab[:10]

['climate',
 'change',
 'is',
 'a',
 'hoax',
 'it',
 'planted',
 'the',
 'troll',
 'the']

In [39]:
top_vocab = Counter(all_vocab)

In [40]:
sorted_v = sorted(top_vocab.items(),key=lambda x:x[1],reverse=True)

In [45]:
sorted_v[0]

('the', 342529)

In [46]:
with open('/Users/yiweiluo/scientific-debates/framing-twitter/data/output/freq_vocab.txt','w') as f:
    for tup_ in sorted_v[:5000]:
        f.write(tup_[0])
        f.write('\n')

In [43]:
with open('/Users/yiweiluo/scientific-debates/framing-twitter/data/output/joint_vocab.txt','r') as f:
    d_vocab = f.read().splitlines()
len(d_vocab)

2041

In [34]:
all_vocab = set(all_vocab)
len(all_vocab)

64605

In [44]:
len(top_vocab)

64605

In [36]:
'cortez' in all_vocab

True

In [37]:
with open('/Users/yiweiluo/scientific-debates/framing-twitter/data/output/vocab.txt','w') as f:
    for v in all_vocab:
        f.write(v)
        f.write('\n')

# Read in results

In [47]:
res = pd.read_csv('cc_kmeans_topics_8.csv')

In [48]:
res.head()

,topic_0,cosine_0,topic_1,cosine_1,topic_2,cosine_2,topic_3,cosine_3,topic_4,cosine_4,topic_5,cosine_5,topic_6,cosine_6,topic_7,cosine_7,indices_in_original
0,2,0.623289,4,0.714146,0,0.764750,3,1.084648,1,1.106662,6,1.133042,5,1.186551,7,1.258481,cc_0
1,1,0.710055,2,0.798882,3,0.958161,5,1.013273,0,1.047892,7,1.120648,6,1.176311,4,1.190092,cc_1
2,2,0.777188,3,0.791400,0,0.871678,1,0.884266,7,1.025838,4,1.141103,6,1.187908,5,1.266636,cc_2
3,0,0.800824,2,0.872168,1,0.923968,4,0.984891,3,1.004107,5,1.112779,7,1.137597,6,1.171677,cc_3
4,1,0.576808,2,0.903400,0,1.016510,4,1.028785,3,1.044035,7,1.163888,5,1.188031,6,1.359631,cc_4


In [49]:
res.topic_0.value_counts()

1    74490
4    54853
5    51599
3    50846
0    50046
2    43190
6    39139
7    36297
Name: topic_0, dtype: int64

In [52]:
res.loc[res.topic_0==0].indices_in_original.values

array(['cc_3', 'cc_55', 'cc_71', ..., 'cc_400410', 'cc_400416',
       'cc_400435'], dtype=object)

In [56]:
int('cc_0'.split('_')[-1])

0

In [58]:
topic_0_quotes = [cc_quotes_df.iloc[int(guid.split('_')[-1])].quote
                  for guid in res.loc[res.topic_0==0].indices_in_original.values]

In [59]:
quotes_by_topic = {0:topic_0_quotes}
for topic in range(1,8):
    quotes_by_topic[topic] = [cc_quotes_df.iloc[int(guid.split('_')[-1])].quote
                  for guid in res.loc[res.topic_0==topic].indices_in_original.values]

In [60]:
topic_0_quotes[:10]

['that Ocasio - Cortez ’s Green New Deal was not enough to solve the climate crisis',
 'that we have the skilled workforce that ’s ready to take on this global challenge  ',
 'many more authority figures being invoked in this way',
 'that the U.S. has just 12 years to take necessary steps or face “ irreparable damage',
 'that climate change is a national emergency',
 'This morning I introduced legislation in the Senate declaring that climate change is a national emergency',
 'climate change is also a “ global emergency',
 'The future of the planet is hanging on our decision about moving forward',
 'Today I introduced legislation with @AOC and @repblumenauer to declare climate change as a national emergency',
 'insisted']

In [61]:
topic_0_quotes[-10:]

['it is in its own interest to become a clean energy superpower',
 'that " failure to act more swiftly " when Williams was absent had contributed to their " anguish and suffering "',
 "that one of our government 's top climate policy advisers , Dieter Helm , is recommending what , to them , would be an eye - wateringly high carbon tax",
 'that as people become more concerned , they will reward companies that are part of the solution',
 'the complexity of our operations requires an even stronger commitment to safety and environment',
 'that these big issues – climate change , global poverty and income inequality in the US – can not be left to governments or NGOs to solve',
 'that the economy operates within environmental limits',
 'that the transition into the post - carbon era must involve not simply " adaptation " and " mitigation " – the stale buzzwords of bureaucracy -',
 'that long - term sustainability for effective local government depends on developing affordable programmes',
 '

In [63]:
quotes_by_topic[1][:10]

['it planted the troll',
 'We ’re not going to be here for much long because of the climate crisis',
 "your next campaign slogan has to be this : ' We got to start eating babies",
 '" We got to start eating babies',
 'people must “ start eating babies',
 ", AOC does n't rule out eating babies",
 'that politicians — who have kicked the can down the road for decades when it comes to climate change —',
 'Biden would be visiting my town',
 'that you ’re not fighting for the people profiting off climate change—',
 'When you talk to scientists about what would be necessary to get to zero carbon emissions , they often talk about people having to change their daily behaviors —']

In [64]:
quotes_by_topic[1][-10:]

['that much will have to change to make this possible',
 'that much will have to change to make this possible',
 'that people are sociable and emotional ,',
 'is to look at the design of the products themselves',
 'that what I have written may jeopardise this work',
 'that the world does',
 'that the stupid runs much deeper than George Monbiot and goes right to the top',
 'that if you need a helping hand , you will find one at the end of your arm',
 'just how bad it was',
 'that what I was trying to say may not have been quite as dotty as they thought']

In [65]:
quotes_by_topic[2][:10]

['climate change is a hoax',
 'the stunt was staged by what is now a far - right pro - Trump conspiracy group that compares climate change activism to “ genocide',
 'Human influence on the climate system is clear',
 'that ’s not such a bad change',
 'the very notion absurd',
 'that it ca n’t possibly be wrong because why would a responsible newspaper put a lie on its front page and why would an economic institution as world renowned as the IMF have produced such a figure if it were n’t accurate',
 'that what this fellow Lord Stern is saying must be true',
 'This is misleading',
 'it ’s clear to see why Todd has been forgotten despite appearing in about 120 films between 1926 and 1935',
 'it hard to believe that a woman who was running a restaurant would n’t have been aware of the risks']

In [66]:
quotes_by_topic[2][-10:]

["that there 's simply not enough research to justify dismissing the possibility of a catastrophe",
 'n’t have that many experts ,',
 "that this is extremely important and that it 's worth missing school for to make decision - makers listen",
 'but ministers swear blind that it is not even on the political agenda ; it is then noted that it might well be on the agenda but is not a serious proposition ; it is later conceded that it is a serious proposition but that it will never be implemented ; after that it is acknowledged that it will be implemented but in such diluted form that it will make no difference to the lives of ordinary people',
 'that it has made such a difference',
 'that the case was n’t sufficiently clear cut to turn it into a major news story',
 'that the current [ threshold ] is subject to review as more',
 'that the ‘ expert ’ class of supposedly trusted authority has in fact had most of its common sense educated out of it at university …  ',
 'that [ climate change ]

In [67]:
quotes_by_topic[3][:10]

['that she was concerned the woman at the town hall',
 'Turns out the woman yelling was a Trump supporter 🤷 🏽\u200d ♀ ️. ” The LaRouche PAC has been running an “ Eat the Children ” campaign modeled on Jonathan Swift ’s',
 'he is a LaRouche PAC –',
 'independent candidate for the US Senate — bragged about Trump giving attention to its “ Eat the Children ” stunt',
 'LaRouchies have been enemies of the green movement for decades',
 'The LaRouchies are now firm Trump supporters — though Putin is their first love',
 'the video doing the rounds this week',
 "he 'll fight for us when he 's opened the door SuperPACs , which would allow any amount of dark fossil fuel money to support his campaign",
 'that he is not up to the task',
 'that Biden , who by most accounts is still in the lead for the Democratic nomination , walked back his campaign ’s opposition to accepting outside help from dark - money super PACs']

In [68]:
quotes_by_topic[3][-10:]

["that Kate 's mother was sat a few rows in front of me",
 'that it pays my wages',
 "By the time I 'd found out what I wanted to know from the primary literature",
 'that all my other efforts to be green - recycling , insulating the house , not driving a giant 4x4 - would be totally wiped out by a couple of holidays by air',
 'why they were n’t answering the questions',
 'he had n’t sent it in the spirit of self - deprecation',
 'that the Climategate emails were going to go public –',
 'it was going to be a big target as soon as it passed the house',
 'that mankind was supposed to be top dog',
 'that when he grew up he would have to face this horror for real']

In [69]:
quotes_by_topic[4][:10]

['Save the planet',
 'that poor Irish families sell their children as food to wealthy landlords',
 'that humans are the dominant cause of " unprecedented " warming of the planet since the 1950s',
 'Scientists are now 95 % certain that humans are the dominant cause of " unprecedented " warming of the planet since the 1950s',
 'that they ’re not taking a single dollar from the merchants of our planet ’s destruction',
 'that a potential " climate apartheid " could fracture the global population , splitting the planet between the wealthy and the rest of the world who will be " left to suffer',
 'n’t believe in catastrophic global warming',
 'that carbon dioxide is a greenhouse gas and that human activities have warmed the planet “ to some unspecified extent ” — both of which are uncontroversial points',
 '“ unprecedented in scale',
 'Their 50-year scenario was positive : “ Humanity already possesses the fundamental scientific , technical and industrial know - how to solve the carbon and cl

In [70]:
quotes_by_topic[4][-10:]

['that the world is interconnected and measure the effects as widely as possible',
 "that the biggest environmental impacts are outside of the manufacturers ' control and usually occur in the supply of raw materials or when the product is being used",
 'he has been a leading voice in drawing the world',
 "However , the planet 's principal problem - from a human point of view - lies with its greenhouse effect",
 'that there was a hunger for a way of grieving ecological loss through ritual',
 'that climate change impacts play a key role in exacerbating the socio - political factors that might speed up this process of communal unravelling',
 'just how preventable the current disaster was : preventable because the causes of these intense , out - of - control bush fires are so widely known',
 'that humans need to respect natural forces',
 'that Bibles are often banned or dangerous to have in other countries',
 'that Bibles are often banned or dangerous to have in other countries']

In [71]:
quotes_by_topic[5][:10]

['Malthusianism is n’t new',
 'But now that Biden is accepting help from super PACs',
 'drive an electric car',
 'Yeah , we ’re going to have to make an investment as a country',
 'oil will be replaced by wind power',
 'My state ’s big industry is going to have to transition into its other big industry',
 'transition into renewable energy jobs —high demand , high skill occupations –',
 'prices should rise to $ 50-$100 by 2030 to give businesses and governments an incentive to lower emissions even when fossil fuels are cheap',
 'poor countries could aim for a lower tax since their economies are more vulnerable',
 'is that we have all of 12 years to transform our energy system away from fossil fuel to energy efficiency and sustainable energy or there will be irreparable damage done to this planet']

In [72]:
quotes_by_topic[5][-10:]

["that most of the world 's fossil fuel reserves need to be left untouched , either forever or at least until we can capture the carbon",
 'that energy density determines environmental impact',
 'there are challenges in cement and concrete production and have been working hard over many years to find solutions',
 'that it is an absolute moral detriment that we are killing people by building roads and by building substandard vehicles',
 'it had been misled by people who were sponsored by the fossil - fuel industry',
 'the Chinese have already invested in iGas and Cuadrilla',
 'that fossil fuel supplies are limited',
 'that pilot whale catches in the Faroe Islands are fully sustainable',
 'that energy density determines environmental impact',
 'that our remaining 1,5 ° carbon budget and ongoing fossil fuel subsidies and investments do n’t add up']

In [73]:
quotes_by_topic[6][:10]

['the world is coming to an end',
 'climate change is a hoax and compares carbon dioxide reduction policies to “ genocide',
 "average global temperatures will rise by 0.5 to 8.6 degrees Fahrenheit and the world 's oceans by 10 to 32 inches by the year 2100",
 '— from rising seas and record hurricanes to coal plants polluting low - income black and indigenous communities',
 'Hi , I ’m Lily from Sunrise',
 'Inskeep asked if O’Rourke supports the oil industry in his state , which provides the state with incredible prosperity , according to the state ’s 2019 State of Energy Report that shows 2018 oil production in Texas totaled a record 1.54 billion barrels ( bbl ) in 2018 , surpassing a previous record of 1.28 billion bbl set in 1973',
 'countries should ramp up the price of carbon emissions to as much as $ 100 per metric ton by 2030 to stop catastrophic global warming',
 'carbon dioxide should be taxed at $ 40-$80 per ton by 2020',
 "PLASTIC POLLUTION IN WORLDS ' OCEANS COULD HAVE $ 2.5 

In [74]:
quotes_by_topic[6][-10:]

['that fertility is a more reliable base for growth than solidity',
 'that not only is the freshly printed money worthless but also most of the manufactured money from the last 20 years',
 'that simply bringing forward a second competition is no longer sufficient to deliver the 10GW or more of CCS anticipated by 2030 , as our recent report showed',
 "that there 's a limited carbon budget",
 'that large industrialised nations need to take the lion \'s share of responsibility for the warming they have caused while allowing developing nations to increase their emissions until a safe stabilisation level - a " carbon democracy " - has been reached',
 'that in the Mediterranean , for example in southern Italy , this would correspond to electricity prices in the range of 15 cents per KWh , something below what the average consumer is paying',
 'that the 67 per cent who had doubts about greenhouse gas theory are almost certainly correct',
 'that they must cast the net for solutions wider than 

In [75]:
quotes_by_topic[7][:10]

['we only have a few months left',
 'the Queen controls',
 'that the LaRouche group has been pulling similar stunts for the past 50 years',
 'that they carried signs like “ Feed Jane Fonda to the whales',
 "Each of the last three decades has been successively warmer at the Earth 's surface than any preceding decade since 1850",
 'You might have seen the video doing the rounds this week :',
 'what happened next',
 'The first , traditional definition is measurable',
 ', STUDY WARNS  ',
 'The Daily Caller reported :']

In [76]:
quotes_by_topic[7][-10:]

['how far gone I was',
 'that during construction there will be disruption to many local residents',
 'that there will be people , at the state and at the city level ,',
 'that Australia has made huge steps in the last 12 months',
 'that he would have to downsize and has now moved to a smaller cottage on Chesil Beach in Dorset',
 'just how much illogical garbage and ill - formed speculation parents of children with these conditions are exposed to',
 'that if they are considering a purchase over the next couple of years , not everyone will be buying this year',
 'that they are sitting on a carbon asset timebomb',
 'that Australia has made huge steps in the last 12 months',
 'that if they are considering a purchase over the next couple of years , not everyone will be buying this year']